In [6]:
import pickle

import cv2
import numpy as np

from kotoamatsukami.src.services.hand_detector.service import HandDetector

# Loading the model

In [13]:
with open("models/logisticregression_model", "rb") as f:
    model = pickle.load(f)

# Labels

In [14]:
labels = {
    0: "click",
    1: "closed",
    2: "down",
    3: "mouse_tracking",
    4: "negative_closed",
    5: "negative_mouse_tracking",
    6: "negative_side",
    7: "negative_up",
    8: "side",
    9: "up"
}

# Normalizing landmakrs

In [15]:
def normalize(z: float, max: float, min: float) -> float:
    norm = (z - min)/(max - min)
    round_norm = round(norm, 3)
    
    return round_norm


def normalize_landmark(landmark: list) -> list:
    xs = [landmark[i][0] for i in range(len(landmark))]
    ys = [landmark[i][1] for i in range(len(landmark))]

    xmax = max(xs)
    ymax = max(ys)
    xmin = min(xs)
    ymin = min(ys)

    norm_landmark = list()

    for x, y in zip(xs, ys):
        norm = [normalize(z=x, max=xmax, min=xmin), normalize(z=y, max=ymax, min=ymin)]

        norm_landmark.append(norm)

    return norm_landmark

# Testing live cam

In [16]:
stream   = cv2.VideoCapture(0)
detector = HandDetector(max_hands = 1, detection_con = 0.8)

i = 0

while True:

    _grabbed, frame = stream.read()

    if cv2.waitKey(1) == ord("x"):
        break

    hands, image = detector.find_hands(frame)

    if len(hands) != 0:
        for hand in hands:
            landmark = hand["landmarks"]

            norm_landmark = normalize_landmark(landmark=landmark)
            
            prediction = model.predict_proba(np.array(norm_landmark).reshape(1, 42))
            i += 1

            if prediction.max() > 0.99:
                print(f"Catch the {labels.get(prediction.argmax())} gesture - {i}")

    cv2.imshow("Video", image)

stream.release()
cv2.destroyAllWindows()

In [18]:
model.predict(np.array(norm_landmark).reshape(1, 42))

array([1])

In [20]:
print(f"Catch the {labels.get(1)} gesture - {i}")

Catch the closed gesture - 484
